# 파이썬의 데코레이터
파이썬의 데코레이터는 함수와 메서드의 기능을 쉽게 수정하기 위한 수단  
이전에는 함수에 변형을 할 때마다 modifier 함수를 사용하여 함수를 호출한 다음 함수를 처음 정의한 것과 같은 이름으로 재할당 해야했음  

e.g) original 이라는 함수가 있고 그 기능을 약간 수정한 modifier라고 하는 함수가 있는 경우 다음과 같이 작성해야 함
```python
def original(...):
    ...
    original = modifier(original)
```
위와 같이 함수를 동일한 이름으로 다시 할당하는 것은 혼란스럽고 오류가 발생하기 쉬우며 번거로움.  
따라서 위의 예제를 아래와 같이 작성할 수 있는 새로운 구문이 등장함  
```python
@modifier
def original(...):
    ...
```
데코레이터는 데코레이터 이후에 나오는 것을 데코레이터의 첫번째 파라미터로 하고 데코레이터의 결과 값을 반환하게 하는 문법  
위의 예제에서 modifier = 데코레이터, original = 데코레이팅된(decorated) 함수 or 래핑된(wrapped) 객체  

처음에는 함수와 메서드를 위해 고안되었으나 실제로는 어떤 종류의 객체에도 적용 가능하므로 함수와 메서드, 제너레이터, 클래스에 적용 가능

## 함수 데코레이터
함수에 데코레이터를 사용하면 어떤 종류의 로직이라도 적용할 수 있음  
- 파라미터의 유효성 검사
- 사전 조건 검사
- 기능 전체 새롭게 정의
- 서명 변경
- 원래 함수의 결과를 캐시

e.g) 도메인의 특정 예외에 대해서 특정 횟수만큼 재시도하는 데코레이터

In [11]:
# decorator_func_1.py
from functools import wraps

class ControlledException(Exception):
    """도메인에서 발생하는 일반적인 예외"""
    
def retry(operation):
    @wraps(operation)
    def wrapped(*args, **kwargs):
        last_raised = None
        RETRIES_LIMIT = 3
        for _ in range(RETRIES_LIMIT):
            try:
                return operation(*args, **kwargs)
            except ControlledException as e:
                logger.info("retrying %s", operation.__qualname__)
                last_raised = e
        raise last_raised
        
    return wrapped

In [18]:
@retry
def run_operation(task):
    """실행 중 예외가 발생할 것으로 예상되는 특정 작업을 실행"""
    return task.run()

여기에서 run_operation 위의 @retry는 실제로 파이썬에서
```python
run_operation = retry(run_operation)
```
을 실행하게 해주는 문법적 설탕일 뿐임

## 클래스 데코레이터
클래스에도 데코레이터를 사용할 수 있는데, 유일한 차이점은 데코레이터 함수의 파라미터로 함수가 아닌 클래스를 받는다는 점  
클래스에서 정의한 속성과 메서드를 데코레이터 안에서 완전히 다른 용도로 변경할 수 있기 때문에 잘못 사용하면 데코레이터는 복잡하고 가독성을 떨어뜨릴 수 있음  

그러나 데코레이터를 클래스에 사용했을 때의 장점 역시 다음과 같이 존재
- 클래스 데코레이터는 코드 재사용과 DRY 원칙의 이점을 공유함. 클래스 데코레이터를 사용하면 여러 클래스가 특정 인터페이스나 기준을 따르도록 강제 가능. 여러 클래스에 적용할 검사를 데코레이터에서 한 번만 하면 됨
- 당장은 작고 간단한 클래스를 생성하고 나중에 데코레이터로 기능 보강 가능
- 어떤 클래스에 대해서는 유지보수 시 데코레이터를 사용해 기존 로직을 훨씬 쉽게 변경 가능. 메타클래스와 같은 방법을 사용해 보다 복잡하게 만드는 것은 권장되지 않음

데코레이터가 유용하게 사용될 수 있는 예제로 모니터링 플랫폼을 위한 이벤트 시스템 예제  
- 각 이벤트의 데이터를 변환하여 외부 시스템으로 보내는 기능
- 이벤트 유형은 데이터 전송 방법에 특별한 점이 있을 수 있음
- e.g 로그인 이벤트에는 자격 증명과 같은 중요한 정보를 숨겨야 함 / timestamp같은 필드는 특별한 포맷으로 표시하므로 변환 필요성 있음

In [20]:
class LoginEventSerializer:
    def __init__(self, event):
        self.event = event
        
    def serialize(self) -> dict:
        return{
            "username": self.event.username,
            "password": "**민감한 정보 삭제**",
            "ip": self.event.ip,
            "timestamp": self.event.timestamp.strftime("%Y-%m-%d %H:%M")
        }
    
class LoginEvent:
    SERIALIZER = LoginEventSerializer
    
    def __init__(self, username, password, ip, timestamp):
        self.username = username
        self.password = password
        self.ip = ip
        self.timestamp = timestamp
        
    def serialize(self) -> dict:
        return self.SERIALIZER(self).serialize()

여기에서는 로그인 이벤트에 직접 매핑할 클래스를 선언함. 이 클래스는 password 필드를 숨기고 timestamp 필드를 포매팅하는 기능이 들어있음  
위의 방법은 시간이 지나면서 시스템이 확장할수록 다음과 같은 문제가 발생  
- 클래스가 너무 많아짐: 이벤트 클래스와 직렬화 클래스가 1대 1로 매핑되어 있으므로 직렬화 클래스가 점점 많아짐
- 이러한 방법은 충분히 유연하지 않음: 만약 password를 가진 다른 클래스에서도 이 필드를 숨기려고 한다면 함수로 분리한 다음 여러 클래스에서 호출해야함. 이는 코드를 충분히 재사용했다고 볼 수가 없음
- 표준화: serialize() 메서드는 모든 이벤트 클래스에 있어야함. 비록 믹스인을 사용해 다른 클래스로 분리할 수 있지만 상속을 제대로 사용했다고 볼 수 없음  

**또 다른 방법은 이벤트 인스턴스와 변형 함수를 필터로 받아서 동적으로 객체를 만드는 것**  
필터를 이벤트 인스턴스의 필드들에 적용해 직렬화하는 방법

In [21]:
from datetime import datetime

def hide_field(field) -> str:
    return "**민감한 정보 삭제**"

def format_time(field_timestamp: datetime) -> str:
    return field_timestamp.strftime("%Y-%m-%d %H:%M")

def show_original(event_field):
    return event_field

class EventSerializer:
    def __init__(self, serialization_fields: dict) -> None:
        self.serialization_fields = serialization_fields
        
    def serialize(self, event) -> dict:
        return {
            field: transformation(getattr(event, field))
            for field, transformation in
            self.serialization_fields.items()
        }
    
class Serialization:
    def __init__(self, **transformations):
        self.serializer = EventSerializer(transformations)
        
    def __call__(self, event_class):
        def serialize_method(event_instance):
            return self.serializer.serialize(event_instance)
        event_class.serialize = serialize_method
        return event_class
    
@Serialization(
    username=show_original, 
    password=hide_field, 
    ip=show_original, 
    timestamp=format_time
)
class LoginEvent:
    def __init__(self, username, password, ip, timestamp):
        self.username = username
        self.password = password
        self.ip = ip
        self.timestamp = timestamp

클래스 데코레이터에 전달된 인수를 읽는 것만으로도 username과 ip는 수정되지 않고, password 필드는 숨겨지고, timestamp는 포매팅 됨을 알 수 있음  


python3.7 이상의 버전에서는 데코레이터를 사용하여 init함수의 템플릿화된 단순 코드를 작성하지 않고도 앞의 예제를 보다 간단히 작성할 수 있음

In [22]:
from dataclasses import dataclass
from datetime import datetime

@Serialization(
    username=show_original, 
    password=hide_field, 
    ip=show_original, 
    timestamp=format_time
)
@dataclass
class LoginEvent:
    username: str
    password: str
    ip: str
    timestamp: datetime